In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
from config import g_key

In [2]:
cameras_file = os.path.join("../Desktop/Archive", "red-light-camera-locations.csv")
speedings_file = os.path.join("../Desktop/Archive","speed-camera-violations.csv")

In [3]:
red_locs_df = pd.read_csv(cameras_file)
del red_locs_df["Third Approach"]
speedings_df = pd.read_csv(speedings_file)
speedings_df.head()

,Address,Camera ID,Date,Violations,X coordinate,Y coordinate,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,10318 S INDIANAPOLIS,CHI120,2019-11-21T00:00:00.000,87,1203645.368,1837056.145,41.707577,-87.529848,"{'latitude': '41.70757690291348', 'human_addre...",47.0,21202.0,49.0,705.0,47.0
1,1110 S PULASKI RD,CHI163,2019-11-21T00:00:00.001,7,1149841.166,1894930.809,41.867603,-87.725383,"{'latitude': '41.86760272243294', 'human_addre...",36.0,21572.0,30.0,98.0,14.0
2,1111 N HUMBOLDT,CHI010,2019-11-21T00:00:00.002,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11144 S VINCENNES,CHI023,2019-11-21T00:00:00.003,6,1166993.613,1830711.217,41.691025,-87.664248,"{'latitude': '41.69102545584918', 'human_addre...",33.0,22212.0,74.0,378.0,42.0
4,11153 S VINCENNES,CHI022,2019-11-21T00:00:00.004,4,1167028.753,1830593.601,41.690702,-87.664122,"{'latitude': '41.690701951255015', 'human_addr...",45.0,22212.0,74.0,378.0,22.0


In [4]:
red_locs_df.head()

,Intersection,First Approach,Second Approach,Date,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,California-Peterson,NB,EB,2009-06-29T00:00:00.000,41.990442,-87.699401,"{'latitude': '41.990442', 'human_address': '{""...",46,4450,20,15,24
1,Central-Addison,SB,EB,2010-11-15T00:00:00.000,41.945877,-87.766754,"{'latitude': '41.945877', 'human_address': '{""...",25,22254,15,361,44
2,Western-63rd,NB,EB,2007-11-12T00:00:00.000,41.779214,-87.683638,"{'latitude': '41.779214', 'human_address': '{""...",44,22257,64,370,2
3,Halsted-79th,NB,EB,2008-04-30T00:00:00.000,41.750643,-87.644043,"{'latitude': '41.750643', 'human_address': '{""...",17,21554,70,556,31
4,Wacker-Lake,NB,SB,2018-02-19T00:00:00.000,41.885700,-87.637000,"{'latitude': '41.8857', 'human_address': '{""ad...",22,14914,38,92,36


In [5]:
cams_per_ward = pd.DataFrame(red_locs_df.groupby("Wards")["Intersection"].count())
violations_per_ward = pd.DataFrame(speedings_df.groupby("Wards")["Violations"].sum())
wards = [1,2,3,4,6,7,8,10,11,13,14,15,16,17,18,19,20,21,22,23,24,26,27,29,30,31,32,33,35,36,38,40,41,44,45,46,48,49,50] 

In [6]:
merge_df = pd.merge(cams_per_ward, violations_per_ward, on="Wards")
merge_df = merge_df.rename(columns={
    "Intersection": "Number of Red Light Cams per Ward",
    "Violations": "Number of Speeding Violations per Ward"
})
merge_df.head()

,Number of Red Light Cams per Ward,Number of Speeding Violations per Ward
Wards,,
1,2,133599
2,2,80740
3,2,76211
4,1,332224
6,1,24005


In [7]:
merge_df["Ward Number"] = wards
merge_df.head()

,Number of Red Light Cams per Ward,Number of Speeding Violations per Ward,Ward Number
Wards,,,
1,2,133599,1
2,2,80740,2
3,2,76211,3
4,1,332224,4
6,1,24005,6


In [8]:
ward_locs = os.path.join("../Desktop/Archive", "ward_offices_locs.csv")
ward_locs_df = pd.read_csv(ward_locs)
ward_locs_df = ward_locs_df.rename(columns={
    "Ward":"Ward Number"
})
ward_locs_df.head()

,Ward Number,Latitude,Longitude
0,1,41.918127,-87.687821
1,2,41.908759,-87.668029
2,3,41.804552,-87.626387
3,4,41.885555,-87.631870
4,5,41.766729,-87.568063


In [9]:
new_merge_df = pd.merge(merge_df, ward_locs_df, on="Ward Number")
new_merge_df.head()

,Number of Red Light Cams per Ward,Number of Speeding Violations per Ward,Ward Number,Latitude,Longitude
0,2,133599,1,41.918127,-87.687821
1,2,80740,2,41.908759,-87.668029
2,2,76211,3,41.804552,-87.626387
3,1,332224,4,41.885555,-87.631870
4,1,24005,6,41.752748,-87.606916


In [10]:
locations = new_merge_df[["Latitude", "Longitude"]]
locations.head()

,Latitude,Longitude
0,41.918127,-87.687821
1,41.908759,-87.668029
2,41.804552,-87.626387
3,41.885555,-87.631870
4,41.752748,-87.606916


In [11]:
gmaps.configure(api_key = g_key)
speed_cams = new_merge_df["Number of Red Light Cams per Ward"].astype(float)

In [12]:
info_box_template = """
<dl>
<dt>Ward</dt><dd>{Ward Number}</dd>
<dt>Number of Red Light Cams</dt><dd>{Number of Red Light Cams per Ward}</dd>
<dt>Number of Speeding Violations</dt><dd>{Number of Speeding Violations per Ward}</dd>
</dl>
"""

In [13]:
x = new_merge_df.to_dict("dict")
x

{'Number of Red Light Cams per Ward': {0: 2,
  1: 2,
  2: 2,
  3: 1,
  4: 1,
  5: 4,
  6: 4,
  7: 1,
  8: 2,
  9: 5,
  10: 2,
  11: 1,
  12: 3,
  13: 6,
  14: 5,
  15: 4,
  16: 3,
  17: 3,
  18: 3,
  19: 6,
  20: 4,
  21: 2,
  22: 3,
  23: 2,
  24: 3,
  25: 5,
  26: 2,
  27: 3,
  28: 1,
  29: 5,
  30: 4,
  31: 7,
  32: 3,
  33: 6,
  34: 3,
  35: 6,
  36: 1,
  37: 4,
  38: 8},
 'Number of Speeding Violations per Ward': {0: 133599,
  1: 80740,
  2: 76211,
  3: 332224,
  4: 24005,
  5: 162942,
  6: 112003,
  7: 117485,
  8: 8872,
  9: 111217,
  10: 349109,
  11: 1305,
  12: 52583,
  13: 37771,
  14: 171539,
  15: 212561,
  16: 223435,
  17: 70620,
  18: 9320,
  19: 221455,
  20: 109605,
  21: 132392,
  22: 68399,
  23: 39464,
  24: 60845,
  25: 212307,
  26: 25237,
  27: 31732,
  28: 88593,
  29: 53345,
  30: 658608,
  31: 161633,
  32: 39022,
  33: 68334,
  34: 77215,
  35: 114740,
  36: 12132,
  37: 89680,
  38: 178772},
 'Ward Number': {0: 1,
  1: 2,
  2: 3,
  3: 4,
  4: 6,
  5: 7,
  6

In [17]:
info_box_template = """
<dl>
<dt>Ward</dt><dd>{Ward Number}</dd>
<dt>Number of Red Light Cams</dt><dd>{Number of Red Light Cams per Ward}</dd>
<dt>Number of Red Light Violations</dt><dd>{Number of Speeding Violations per Ward}</dd>
</dl>
"""

In [18]:
fig = gmaps.figure()
heat_map = gmaps.heatmap_layer(locations, weights= speed_cams, dissipating=False, max_intensity=1, point_radius=0.01)
fig.add_layer(heat_map)
cam_info = [info_box_template.format(**row) for index, row in new_merge_df.iterrows()]
markers = gmaps.marker_layer(locations, info_box_content= cam_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))